In [10]:
%matplotlib notebook

import time
from importlib import reload

from matplotlib import pyplot as plt
from matplotlib import colors, cm
import numpy as np

import qcodes
import qcodes as qc

from pytopo.qctools.instruments import add2station, create_inst
from pytopo.awg_sequencing import awg_tools

# Init station

In [2]:
%run -i D:/OneDrive/BF2/Code/BF2/BF2/init_station.py
%run -i D:/OneDrive/BF2/Code/BF2/BF2/init_station_cqed.py

Connected to: TEKTRONIX AWG5014C (serial:B051388, firmware:SCPI:99.0 FW:4.6.0.7) in 0.60s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110512, firmware:3.1.19.15-3.50.124.67) in 0.05s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110531, firmware:3.1.19.15-3.50.124.67) in 0.05s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110509, firmware:3.1.19.15-3.50.124.67) in 0.05s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110869, firmware:3.1.19.26-3.50.124.73) in 0.05s
Initialized IVVI-rack in 0.06s
Connected to: YOKOGAWA GS210 (serial:91T926459, firmware:2.02) in 0.06s
Connected to: Rigol Technologies DG1062Z (serial:DG1ZA195006391, firmware:03.01.12) in 0.06s


[yoko(GS200)] Snapshot: Could not update parameter: voltage_range
[yoko(GS200)] Snapshot: Could not update parameter: voltage
[fg_burst(DG1062Burst)] Snapshot: Could not update parameter: period
[fg_burst(DG1062Burst)] Snapshot: Could not update parameter: period


## instrument defaults

In [3]:
station.hetsrc.IF(25e6)
station.hetsrc.frequency(4.8e9)

## Acquisition controllers

In [7]:
from pytopo.rf.alazar import acquisition_controllers; reload(acquisition_controllers)
from pytopo.rf.alazar import softsweep as ss; reload(ss)

# controller for 'simple' post-acquisition demod
post_iq_acq = instools.create_inst(
    acquisition_controllers.PostIQCtl, 
    'post_iq_acq', 'alazar',
    allocate_samples=5e8, 
    force_new_instance=True,
)
station = add2station(station, post_iq_acq)
post_iq_acq.demod_frq(25e6)
post_iq_acq.reference_channel(1)

# controller that can step frequencies during acquisition.
softsweep_ctl = instools.create_inst(
    ss.SoftSweepCtl,
    'softsweep_ctl', 'alazar',
    allocate_samples=5e8, 
    force_new_instance=True)
station = add2station(station, softsweep_ctl)
softsweep_ctl.demod_frq(25e6)
softsweep_ctl.reference_channel(1)

Instrument post_iq_acq already exists.
Allocating 500000000 elements (2000.0 MB, dtype: <class 'numpy.int32'>)
Instrument softsweep_ctl already exists.
Allocating 500000000 elements (2000.0 MB, dtype: <class 'numpy.int32'>)


## AWG sequences

In [8]:
from pytopo.rf.alazar.awg_sequences import TriggerSequence

TriggerSequence.chan_map = {
    1 : ['pulse', 'ats_trigger', 'ro_trigger'],
}
TriggerSequence.chan_settings = {
    1 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
    2 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
}

# Resonator spec

In [11]:
fvals = np.arange(4600, 4700, 0.5) * 1e6

softsweep_ctl.next_point_trigger_func = lambda: awg_tools.trigger_awg_when_ready(station.awg)
softsweep_ctl.settling_time = 0.01

ds = ss.measure_triggered_softsweep(
    softsweep_ctl, station.hetsrc.frequency, fvals, integration_time=20e-3, channel=0,
)

Starting experimental run with id: 27
Setup capture: 3194.88 MB total
 * Buffers: 8000 (0.39936 MB/buffer) | (Allocated buffers: 40 = 15.9744 MB)
 * Records: 1 (0.39936 MB/record)
 * Samples: 49920 (= 499.2 us)
 * Channels: 2


In [12]:
reps = qcodes.Parameter('repetition', set_cmd=None)

ds = ss.measure_triggered_softsweep_vs_parameter(
    softsweep_ctl, station.hetsrc.frequency, fvals, integration_time=5e-3, channel=0,
    parameter=reps, values=np.arange(20.),
)

Starting experimental run with id: 28
Setup capture: 798.72 MB total
 * Buffers: 2000 (0.39936 MB/buffer) | (Allocated buffers: 10 = 3.9936 MB)
 * Records: 1 (0.39936 MB/record)
 * Samples: 49920 (= 499.2 us)
 * Channels: 2


# Scratch